In [1]:
import sys

In [2]:
sys.path.append("/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/my_engine/chess-sauce")

In [3]:
import my_engine
import chess
import stockfish
import random
from line_profiler import LineProfiler
import pandas as pd

In [4]:
sf = stockfish.Stockfish(path="/Users/diegoesparza/Downloads/stockfish/stockfish-macos-x86-64")

In [5]:
# this is mostly good. one ~flaw is that it could leave the position in a state
# where white has to play a specific move to save a piece from capture. worth
# thinking about whether this would cause the same issue as hanging piece /
# tactic. maybe not because, since white can move the piece out of harms way, it
# won't affect stockfish eval. and we don't expect model to pick up on attacks
# necessarily.
#
# we could explore until various heuristics are met:
# 1. second best piece to move is close in centipawn to best move
# 2. best non capture move is close in centipawn to best move
def gen_positions_from(b, sf, positions_to_generate, depth):
    if depth == 0:
        next_depth = 0
        if positions_to_generate != 1:
            raise ValueError(positions_to_generate)
        fen = b.fen()
        sf.set_fen_position(fen)
        moves = sf.get_top_moves(1)
        if sf.will_move_be_a_capture(moves[0]["Move"]) != stockfish.Stockfish.Capture.NO_CAPTURE:
            moves_to_explore = [chess.Move.from_uci(move["Move"]) for move in moves]
        else:
            print(fen)
            return None
    else:
        next_depth = depth-1
        moves = list(b.legal_moves)
        if positions_to_generate < len(moves):
            moves = random.sample(moves, positions_to_generate)
        moves_to_explore = moves

    if len(moves_to_explore) == 0:
        fen = b.fen()
        raise ValueError(f"reached end: {fen}")
    pos_per = positions_to_generate // len(moves_to_explore)
    rem = positions_to_generate - pos_per * len(moves_to_explore)
    for idx, move in enumerate(moves_to_explore):
        next_positions_to_generate = pos_per
        if idx < rem:
            next_positions_to_generate += 1
        b.push(move)
        gen_positions_from(b, sf, next_positions_to_generate, next_depth)
        b.pop()
        
def gen_positions(sf, opening_fens, positions_to_generate, depth):
    if positions_to_generate < len(opening_fens):
        opening_fens = random.sample(opening_fens, positions_to_generate)
        
    pos_per = positions_to_generate // len(opening_fens)
    rem = positions_to_generate - pos_per * len(opening_fens)
    for idx, fen in enumerate(opening_fens):
        next_positions_to_generate = pos_per
        if idx < rem:
            next_positions_to_generate += 1
        b = chess.Board(fen)
        try:
            gen_positions_from(b, sf, next_positions_to_generate, depth)
        except ValueError as e:
            print(f"Error from {fen}: {e}")

In [9]:
positions_to_generate = 10
depth = 8

lp = LineProfiler()
lp.add_function(gen_positions_from)
lp.add_function(gen_positions)
opening_fens = !cat "/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/kaggle_fens/strong_openings"
lp.runcall(gen_positions, sf, opening_fens, positions_to_generate, depth)
lp.print_stats()

rn2kb1r/1p2qp1p/p2p2p1/4P2P/4P3/1PN5/P1PB2BP/R2bK2R b KQkq - 0 13
rnb3nr/1pp2k1p/3p2p1/p2Pp3/N1P1Pp1q/3B1P1P/PP4P1/R2Q1KNR w - - 5 13
1rbq1rk1/ppp1bppp/8/3p4/3Pn3/P1PBBNP1/P3RP1P/RN1Q2K1 b - - 0 13
rn3rk1/pbq1p1bp/3pBnp1/1pp5/3P1PP1/2P4P/PP6/RNBQK1NR b KQ - 0 13
rq3rk1/pp2bppp/2npb3/7n/4P3/2NQ1N2/PP3PPP/R1B2RK1 w - - 4 13
4k2r/ppq2ppp/2n1pn2/3p1b2/1P1P4/1N3NP1/P1P1BP1P/R2Q1RK1 w k - 0 14
rn3rk1/p1p2ppp/3q4/1p1p4/P2P2Bb/2N1P3/1P3PPP/1R1QK2R b K - 0 13
r1bqr1k1/p4ppp/1pn1p3/b2p3n/2Pp4/P3PNP1/1PB1NP1P/R1B1QRK1 w - - 2 13
rnb2rk1/p2nq1b1/3pp1p1/1Pp4p/4PP2/3B1NKP/PP4P1/RNBQ1R2 b - - 0 14
Bn1qbrkb/p2n1p1p/6p1/3p4/Q2P3P/4PN2/PP2KPP1/R1B4R b - - 0 14
Timer unit: 1e-09 s

Total time: 3.14365 s
File: /var/folders/j2/1qmnvcts25x7hrnmfmyczwg40000gn/T/ipykernel_92210/410664942.py
Function: gen_positions_from at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def gen_positions_from(b, sf, positions_to_generate, depth):
     8   

In [13]:
sf.set_position()
for i in range(10):
    print(sf.get_top_moves(4))

[{'Move': 'e2e4', 'Centipawn': 25, 'Mate': None}, {'Move': 'd2d4', 'Centipawn': 23, 'Mate': None}, {'Move': 'g1f3', 'Centipawn': 18, 'Mate': None}, {'Move': 'e2e3', 'Centipawn': 18, 'Mate': None}]
[{'Move': 'd2d4', 'Centipawn': 26, 'Mate': None}, {'Move': 'e2e4', 'Centipawn': 24, 'Mate': None}, {'Move': 'g1f3', 'Centipawn': 18, 'Mate': None}, {'Move': 'e2e3', 'Centipawn': 13, 'Mate': None}]
[{'Move': 'e2e4', 'Centipawn': 26, 'Mate': None}, {'Move': 'd2d4', 'Centipawn': 22, 'Mate': None}, {'Move': 'g1f3', 'Centipawn': 21, 'Mate': None}, {'Move': 'e2e3', 'Centipawn': 13, 'Mate': None}]
[{'Move': 'e2e4', 'Centipawn': 22, 'Mate': None}, {'Move': 'd2d4', 'Centipawn': 21, 'Mate': None}, {'Move': 'g1f3', 'Centipawn': 20, 'Mate': None}, {'Move': 'c2c4', 'Centipawn': 16, 'Mate': None}]
[{'Move': 'd2d4', 'Centipawn': 23, 'Mate': None}, {'Move': 'e2e4', 'Centipawn': 21, 'Mate': None}, {'Move': 'g1f3', 'Centipawn': 20, 'Mate': None}, {'Move': 'e2e3', 'Centipawn': 11, 'Mate': None}]
[{'Move': 'e2e4